In [1]:
import pandas as pd
import json
import logLibrary as lib
import os
import sys
import base64
import shutil
from github import Github
from github import GithubException

In [2]:
def get_sha_for_tag(repository, tag):
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha
    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [3]:
def download_directory(repository, sha, server_path,saving_direct):
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    #if os.path.exists(server_path):
    #    shutil.rmtree(server_path)
    #os.makedirs(server_path)
    print(repository)
    contents = repository.get_dir_contents(server_path, ref=sha)
    for content in contents:
        print("Processing %s" % content.path)
        if content.type == 'dir':
            if os.path.exists(saving_direct+content.path):
                shutil.rmtree(saving_direct+content.path)
            os.makedirs(saving_direct+content.path)
            download_directory(repository, sha, content.path,saving_direct)
        else:
            try:
                path = content.path
                if ".java" in path:
                    file_content = repository.get_contents(path, ref=sha)
                    file_data = base64.b64decode(file_content.content)
                    file_out = open(saving_direct+content.path, "wb+")
                    file_out.write(file_data)
                    file_out.close()
            except (GithubException, IOError) as exc:
                print('Error processing %s: %s', content.path, exc)

In [4]:
def usage():
    """
    Prints the usage command lines
    """
    print("usage: gh-download --token=token --org=org --repo=repo --branch=branch --folder=folder")

In [5]:
def main():
    """
    Main function block
    """
    file = r"C:\CodeAnalytics\config\gitdict.json"
    d = open(file)
    data = json.load(d)
    df = pd.DataFrame.from_dict(data,orient='index')
    df = df.reset_index()
    
    df.rename(columns = {"index": "Key"}, 
          inplace = True)
    pattern = r'(.*)\/(.*)\/(.*)\/(.*)'
    df['Organisation'] = df.Key.str.extract(pattern).get(0)
    df['Repository'] = df.Key.str.extract(pattern).get(1)
    df['Branch'] = df.Key.str.extract(pattern).get(2)
    
    #df = pd.read_csv(r"C:\sunil\csv files\gitdirectories.csv")
    for index,row_data in df.iterrows():
        commit = row_data['commit']
        Organisation = row_data['Organisation']
        Repository = row_data['Repository']
        Branch = row_data['Branch']
        
        
        json_obj = lib.json_reader(r"C:\CodeAnalytics\config\gitTempConfig.json")
        
        token = json_obj['token']
        folder = json_obj['server_path']
        saving_direct = json_obj['saving_directory']
        saving_direct = saving_direct+Repository+'/'
        #github = Github(token)
        github = Github(base_url="https://wwwin-github.cisco.com/api/v3",login_or_token=token)
        organization = github.get_organization(Organisation)
        repository = organization.get_repo(Repository)
    #repository = github.get_user(org).get_repo(repo)
        sha = get_sha_for_tag(repository, Branch)
        download_directory(repository, sha, folder,saving_direct)

In [6]:

main()

Repository(full_name="mobile-cnat-policy/policy-products")


<ipython-input-3-61b81e43ed81>:10: DeprecationWarning: Call to deprecated method get_dir_contents. (
        Repository.get_dir_contents() is deprecated, use
        Repository.get_contents() instead.
        )
  contents = repository.get_dir_contents(server_path, ref=sha)


Processing .gitignore
Processing .helmignore
Processing Jenkinsfile.smi
Processing README.md
Processing component.properties
Processing deploy.sh
Processing deploy.yaml
Processing helmfile.yaml
Processing init
Processing undeploy.sh
Repository(full_name="mobile-cnat-infrastructure/cnat-cps-infrastructure")
Processing .gitignore
Processing .helmignore
Processing .qa-build
Repository(full_name="mobile-cnat-infrastructure/cnat-cps-infrastructure")
Processing .qa-build/Jenkinsfile.snapshot
Processing .qa-build/Jenkinsfile.snapshot.bak
Processing Chart.yaml
Processing Jenkinsfile.smi
Processing README.md
Processing component.properties
Processing init
Processing meta
Repository(full_name="mobile-cnat-infrastructure/cnat-cps-infrastructure")
Processing meta/main.yml
Processing requirements-chart.yml
Processing values.yaml
Repository(full_name="mobile-cnat-infrastructure/cps-admin-db")
Processing .gitignore
Processing .helmignore
Processing .qa-build
Repository(full_name="mobile-cnat-infrastr